# Accretion disk

In this tutorial we will set up an accretion disk, which is a radiation source separate from the star, and add its contribution to the signal. The accretion disk we will use here (Mitsuda et al. 1984, Makishima et al. 1986) is a superposition of multicolor blackbody rings. It does not include relativistic effects and spectral hardening.

## Setting up the disk model

We follow here the description given by Makishima et al. (1986). The disk is parametrized by an inner cut-off radius, $R_\rm{in}$, a temperature at this radius, $T_\rm{in}$, and an outer radius, $R_\rm{out}$. The outer rings will contribute much less to the X-ray spectrum than inner rings due to being much colder, so the exact value of $R_\rm{out}$ is much less consequential than $R_\rm{in}$. For each ring, the temperature at some radius $T(r)$ is determined by the (gravitational) energy release associated with accretion towards a central object of mass $M_*$. It is expressed as:

$$T(r) = \left(\frac{3 G \dot{M} M_*}{8 \pi \sigma r^3}\right)^{1/4},$$

where $G$ is the gravitational constant, $\dot{M}$ is the accretion rate and $\sigma$ is the Stefan-Boltzmann constant. The surface area $dA$ of an infinitesimal ring between between $r-dr$ and $r$ is given by $2 \pi r dr$. The specific luminosity $dL_\rm{ring}(E)$ (energy/time/energy) of two hemispheres becomes

$$dL_\rm{ring}(E) = 4 B(E,T(r)) dA \int d\Omega= 8 \pi B(E,T(r)) r dr \int d\Omega = 8 \pi^2 B(E,T(r)) r dr.$$ 

Here the integral of the solid angle over one hemisphere evaluates to $\pi$. We multiply by 2 for the two sides of the disk and another factor of 2 appears since we define the $B(E, T)$, as half of the blackbody spectral radiance,  

$$B(E,T) = \frac{E^3}{c^2h^2}\left[\exp{\left(\frac{E}{k_\rm{B}T}\right)-1}\right]^{-1}. $$

Below we give the python definition of $B(E, T)$:

In [3]:
def B_E(self, E, T):
    '''
    Half the energy radiance of a blackbody. If the the photon energy is much higher than kT, 
    the radiance is set to zero and ignores an error

    parameters:
        E in keV
        T in keV

    returns:
        B_E in keV/s/keV/cm^2/sr (you will integrate over keV)
    '''
    with np.errstate(over='ignore'):
        safe_ET = np.float128(E / T)
        exp_safe_ET = np.exp(safe_ET)
    B = np.where(np.isinf(exp_safe_ET), 0, E**3 / (_h_keV**3 * _c_cgs**2) / (exp_safe_ET - 1))
    return B

The specific flux of the disk $f_\rm{disk}(E)$ (energy/time/length$^2$/energy) is the integral of the contribution of each ring from $R_\rm{in}$ to $R_\rm{out}$. But, this can be conveniently expressed as an integral over $dT$:


$$f_\rm{disk}(E) = \frac{\cos{i} \int dL_\rm{ring}}{4\pi D^2}=\frac{2 \pi \cos{i}}{D^2}\int_{R_\rm{in}}^{R_\rm{out}} r B(E,T(r)) dr = \frac{8\pi R_\rm{in}^2 \cos{i}}{3 D^2 T_\rm{in}}\int_{T_\rm{out}}^{T_\rm{in}}\Big(\frac{T}{T_\rm{in}}\Big)^{-11/3}B(E,T) dT.$$

Where $D$ is the distance and we introduced $\cos{i}$ to scale the flux to account for the viewing angle.

In [8]:
    def get_f_disk(self, energies):
        """ Evaluate f_disk(E).
        
        Parameters
        ----------
        energies[keV]
        
        Returns
        -------
        f_disk [keV/s/cm^2/keV]
        
        """
        
        T_in = self['T_in']
        K_disk = self['K_disk']

        # KbT in keV
        T_in_keV = k_B_over_keV * pow(10.0, T_in)
        
        T_out_keV = T_in_keV*1e-1
        
        epsrel = 1e-4

        f_disk_array = np.array([]) #photons/s/cm^2/sr or keV/s/cm^2/sr 
        for energy in energies:
            f_disk_value = self.l_disk(energy, T_in_keV, T_out_keV, spectral_radiance, epsrel) 
            f_disk_array=np.append(f_disk_array,f_disk_value)
        
        # K_disk is cos_i*R_in^2/D^2 in (km / 10 kpc)^2.
        # (1 km / 10 kpc)^2 = 1.0502650e-35 [ cm/cm ]
        
        f_disk_array *=K_disk*8*np.pi/3*1.0502650e-35 # photons/s/cm^2/energy_bin
        
        #print("f_disk_array:",f_disk_array)
        #exit()
        
            # Apply Interstellar if not None
        # if attenuate:
        #     if self.interstellar is not None:
        #         self.interstellar(energies, f_disk_array) # bkg is overwritten here
        
        return f_disk_array

Optionally, to obtain the bolometric flux (energy/length^2/time) of the disk one needs to integrate over energy:

$$F_\rm{disk} = \int f_\rm{disk}(E) dE.$$

To obtain the photon flux for the disk instead of energy flux, one can divide $f_\rm{disk}(E)$ by the photon energy.